In [7]:
import pandas as pd
import numpy as np

In [8]:
defaults = pd.read_csv('../../rawdata/weather/weather.defaults')
defaults = defaults.columns.values

def removeErrorValues(row):
    
    if(row > 20 and str(row) in defaults):
        return 0
    else:
        return row

In [46]:
cols=[0,11, 28, 29]
# weatherDFrame_header = pd.read_csv('../../rawdata/weather/weather.header')
weatherDFrame_header = pd.read_csv('../../rawdata/weather/weather.header', usecols = cols)
weatherDFrame = pd.read_csv('../../rawdata/weather/weather', usecols = cols, names=weatherDFrame_header.columns.values)
# weatherDFrame['datetime'] = pd.to_datetime(weatherDFrame['time'],unit='s')
# weatherDFrame['datetime'].dt.tz_localize('utc').dt.tz_convert('US/Central')

# weatherDFrame.sort_values(by=['datetime'], inplace=True)
# weatherDFrame.set_index(['datetime'], inplace = True)
# weatherDFrame.drop(['time'], axis=1, inplace=True)

In [45]:
weatherDFrame.iloc[3000:3100]

,time,Spd[Wind],Pr[PrecipHourly1],Amt[PrecipHourly1],I[PrecipHourly1],Q[PrecipHourly1],Pr[PrecipHourly2],Amt[PrecipHourly2],I[PrecipHourly2],Q[PrecipHourly2]
3000,1269299160,7.7,99,999.9,9,9,99,999.9,9,9
3001,1269301740,6.7,99,999.9,9,9,99,999.9,9,9
3002,1269301860,7.7,1,0.0,9,5,99,999.9,9,9
3003,1269304800,7.7,1,0.0,2,1,99,999.9,9,9
3004,1269305460,8.8,1,0.3,9,5,6,0.3,9,1
3005,1269306000,8.7,6,0.3,9,1,99,999.9,9,9
3006,1269307080,8.8,1,3.8,9,1,99,999.9,9,9
3007,1269309060,8.8,1,11.9,9,5,99,999.9,9,9
3008,1269309900,8.2,1,7.1,9,1,99,999.9,9,9
3009,1269312300,8.2,1,8.1,9,1,99,999.9,9,9


In [31]:
### cleaning default values
weatherDFrame['Spd[Wind]'] = weatherDFrame.apply(lambda row: removeErrorValues(row['Spd[Wind]']), axis=1)

In [33]:
weatherDFrame.to_pickle('../../data/weather/weather.pkl')